**Without code**

```jupyter nbconvert decision_engine_presentation.ipynb --to slides --no-input --post serve```

**With code**

```jupyter nbconvert decision_engine_presentation.ipynb --to slides --post serve```

**Long slides with code**

```jupyter nbconvert decision_engine_presentation.ipynb --to slides --SlidesExporter.reveal_scroll=True --post serve```


In [1]:
from decision_engine import MinimalSupplierSelectionModel, SupplierSelectionModel
import random

In [2]:
def generate_supplier_selector_variables(min_price=100, max_price=500,
                                         min_demand=50, max_demand=300,
                                         n_suppliers=15, n_parts=50, 
                                         n_years=4, print_data=True):

    price = []
    for supplier in range(n_suppliers):
        s = []
        for _ in range(n_parts):
            y = []
            for _ in range(n_years):
                y.append(random.randint(0, 500))
            s.append(y)
        price.append(s)

    demand = []
    for _ in range(n_parts):
        y = []
        for _ in range(n_years):
            y.append(random.randint(0, 300))
        demand.append(y)

    capacity = []
    for _ in range(n_suppliers):
        y = []
        for _ in range(n_years):
            y.append(random.randint(n_parts-2, n_parts))
        capacity.append(y)

    share = []
    for _ in range(n_suppliers):
        s = []
        for _ in range(n_parts):
            s.append(random.randrange(75, 101, 5))
        share.append(s)
        
    supplier_transfer_limit = []
    for _ in range(n_suppliers):
        supplier_transfer_limit.append(random.randint(n_parts-2, n_parts))

    if print_data:
        print(f'number of suppliers: {n_suppliers}')
        print(f'number of parts: {n_parts}')
        print(f'price: {price}')
        print(f'demand: {demand}')
        print(f'capacity: {capacity}')
        print(f'share: {share}')
        print(f'supplier transfer limit: {supplier_transfer_limit}')

    return price, demand, capacity, share, supplier_transfer_limit

# Decision Engine
## Introduction

# Typical problem statement

- Rolls-Royce need to procure approximately 1000 unique forged metal parts for the Trent XWB engine.
- There are 30 suppliers globally that have the required skills and equipment to manufacture the required parts.
- Rolls-Royce provide all 30 suppliers with the information required to price and submit a bid to manufacture any of the parts.
- Rolls-Royce receive the bids and must then decide who is awarded the manufacturing contract for every part

## Constraints

- At first this might appear to be trivial (select the cheapest bid for every part), however there are a number of constraints that must be considered
    - There is a limit on the number of parts a supplier can manufacture
    - Manufacturers based in certain countries (e.g. China) are limited to manufacturing a percentage of the total required volume of a part

## Combinatorial optimisation

- The problem is no longer trivial and is an example of a combinatorial optimisation problem 
- A combinatorial optimisation problem is the act of trying to find out the value (combination) of variables that optimises an index (value) from among many options under various constraints.
- The problem can be solved using constraint programming (CP) techniques

# Minimal problem

The supplier selection problem reduced to its most basic form

## Demand

A buyer is tasked with sourcing four parts for the manufacturing of a jet
engine. The demand for each part is defined by the below table.

| Part 1 | Part 2 | Part 3 | Part 4 |
| ------ | ------ | ------ | ------ |
| 300 | 20 | 150 | 80 |

## Price

There are two suppliers from which the buyer can source the required parts. The price quoted by each supplier for every part is detailed in the following table.

| Supplier | Part 1 | Part 2 | Part 3 | Part 4 |
| -------- | ------ | ------ | ------ | ------ |
| 1 | £60 | £605 | £95 | £75 |
| 2 | £50 | £615 | £98 | £60 |


## Manufacturing capacity

Each supplier has constraints on manufacturing - i.e. they only have the capacity to manufacture $x$ number of parts. The manufacturing capacity of both suppliers is specified below.

| Supplier | Parts limit | 
| -------- | ----------- |
| 1 | 2 |
| 2 | 3 |

## Objective and constraints

The **objective** is to minimise the total cost of procuring parts subject to the following constraints:

- Constraint 1 - a supplier cannot be assigned more parts than that defined by their manufacturing capacity
- Constraint 2 - the total manufactured volume of a part must be equal to demand

## Solution

The buyer must procure parts from suppliers so that the total cost is minimised. The optimal solution (i.e. the solution that minimises cost) is detailed in the following table.


In [3]:
price = [[60, 605, 95, 75],
         [50, 615, 98, 60]]

demand = [300, 20, 150, 80]

capacity = [2, 3]

supplier_selection = MinimalSupplierSelectionModel(price, demand,
                                                   capacity)

In [4]:
supplier_selection.minimise_cost()


Optimal solution found: cost - £46,150.00

Supplier 0
----------
Part 0 volume: 0
Part 1 volume: 20
Part 2 volume: 150
Part 3 volume: 0

Supplier 1
----------
Part 0 volume: 300
Part 1 volume: 0
Part 2 volume: 0
Part 3 volume: 80


# Additional constraints

## Maximum share

The buyer might have to consider an additional constraint on the maximum share of a part that can be assigned to a supplier. For example, manufacturers in China can only be assigned 30\% of the volume for certain parts.

| Supplier | Part 1 | Part 2 | Part 3 | Part 4 |
| -------- | ------ | ------ | ------ | ------ |
| 1 | 100% | 100% | 70% | 100% |
| 2 | 30% | 100% | 30% | 75% |

## Solution

Solving the same problem but with constraints on the maximum share, the cost rises from **£46,150** to **£49,662**

In [5]:
price = [[60, 605, 95, 75],
         [50, 615, 98, 60]]

demand = [300, 20, 150, 80]

capacity = [2, 3]

share = [[100, 100, 30, 100],
         [80, 100, 70, 100]]

supplier_selection = MinimalSupplierSelectionModel(price, demand, 
                                                   capacity, share)

In [6]:
supplier_selection.minimise_cost()


Optimal solution found: cost - £49,662.00

Supplier 0
----------
Part 0 volume: 300
Part 1 volume: 0
Part 2 volume: 46
Part 3 volume: 0

Supplier 1
----------
Part 0 volume: 0
Part 1 volume: 20
Part 2 volume: 104
Part 3 volume: 80


### Without maximum share constraint

| Supplier | Part 1 | Part 2 | Part 3 | Part 4 |
| -------- | ------ | ------ | ------ | ------ |
| 1 | 0 | 20 | 150 | 0 |
| 2 | 300 | 0 | 0 | 80 |

### With maximum share constraint

| Supplier | Part 1 | Part 2 | Part 3 | Part 4 |
| -------- | ------ | ------ | ------ | ------ |
| 1 | 300 | 0 | 46 | 0 |
| 2 | 0 | 20 | 104 | 80 |

# Going beyond the minimal problem

The price of parts is a function of time and changes year to year. For example, the price of parts will increase over time due to inflationary effects. Conversely the price might decrease due to learning effects. This is where the cost of production per unit decreases over time as suppliers become more familiar with the production process, hence leading to improvements in their efficiency level. The buyer might wish to switch the supplier of a part as time progresses but there will also be an associated transfer cost.

In [7]:
price = [[[60, 62, 64], [605, 610, 615], [95, 96, 97], [75, 75, 75]],
         [[50, 55, 60], [615, 610, 605], [98, 97, 96], [60, 70, 80]]]

demand = [[300, 310, 320], [20, 30, 40], [150, 145, 130], [80, 80, 80]]

capacity = [[4, 4, 4], [3, 3, 3]]

share = [[100, 100, 30, 100],
         [80, 100, 70, 100]]

supplier_transfer_limit = [1, 2]

supplier_selection = SupplierSelectionModel(price, 
                                            demand, 
                                            capacity, 
                                            supplier_transfer_limit, 
                                            3, 
                                            share)

# supplier_selection.set_volume_constraint(1, 2, 0, 100)  # Supplier 1, Part 2, Year 0
# supplier_selection.set_volume_constraint(1, 3, 2, 70)    # Supplier 1, Part 3, Year 2

In [8]:
supplier_selection.minimise_cost()


Optimal solution found: cost - £162,900.00


Part  0              0         1         2
-------

Supplier  0:        60        62        64
                     0         0         0
                     0         0         0
                     0         0         0

Supplier  1:        50        55        60
                   300       310       320
                     1         1         1
                     0         0         0


Part  1              0         1         2
-------

Supplier  0:       605       610       615
                    20        30         0
                     1         1         0
                     0         0         0

Supplier  1:       615       610       605
                     0         0        40
                     0         0         1
                     0         0         1


Part  2              0         1         2
-------

Supplier  0:        95        96        97
                   150       145         0
                  

In [9]:
price, demand, capacity, share, supplier_transfer_limit = generate_supplier_selector_variables(n_suppliers=5, 
                                                                                               n_parts=10,
                                                                                               n_years=10,
                                                                                               print_data=False);

supplier_selection = SupplierSelectionModel(price, 
                                            demand, 
                                            capacity, 
                                            supplier_transfer_limit, 
                                            10, 
                                            share)

In [10]:
supplier_selection.minimise_cost()


Optimal solution found: cost - £1,419,818.00


Part  0              0         1         2         3         4         5         6         7         8         9
-------

Supplier  0:       145         2        24       365       473       201       453       313       228        98
                     0         0         0         0         0         0         0         0         0       149
                     0         0         0         0         0         0         0         0         0         1
                     0         0         0         0         0         0         0         0         0         1

Supplier  1:       387       151       361        67       498        65       268         4         4       273
                     0         0         0         0         0         0         0       127         0         0
                     0         0         0         0         0         0         0         1         0         0
                     0         0      